Setup the environment to run the ML and AutoML jobs within our Azure ML workspace.

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity1")

#ws = Workspace(subscription_id, resource_group, workspace_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Creat the cluster to be used to run our ML jobs/experiments.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "udacitycluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Setup the parameters for our ML experiments using HyperDrive.

Azure Machine Learning allows for hyperparameter training through Hyperdrive experiments. 

This process launches multiple child runs, each with a different hyperparameter configuration. After all, runs are complete; the best model can be evaluated and registered to the Azure Machine Learning Studio.

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
from azureml.core.conda_dependencies import CondaDependencies

# Specify parameter sampler
ps = RandomParameterSampling( {  
    "--C": choice(0.001, 0.01, 0.1, 1, 10, 20, 50, 100, 200, 500, 1000),
    "--max_iter": choice(100, 200, 300)
    }
)

# Specify a Policy: check the job every 2 iterations and if the primary metric falls outside the top 50% range, 
# Azure ML should terminate the job
policy = BanditPolicy(evaluation_interval=5, slack_factor=0.1)

packages = CondaDependencies.create(
    pip_packages=['azureml-defaults','scikit-learn']
)

if "training" not in os.listdir():
    os.mkdir("./training")

# https://ml.azure.com/fileexplorerAzNB?wsid=/subscriptions/a022d83d-6229-4b5e-b039-b680692436b5/resourceGroups/azureai-test-east/providers/Microsoft.MachineLearningServices/workspaces/project-azureai-patton-test-east&tid=41b3409a-40cf-45e8-8201-ddfa1984ab3d&activeFilePath=Users/aspatton01/conda_dependencies.yml
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
#sklearn_env = Environment("sklearn-env")
#sklearn_env.python.conda_dependencies = packages

src = ScriptRunConfig(source_directory="./",
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=8,
                                     max_concurrent_runs=4)


The HyperDrive runs are submitted with details displayed as they run.

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

# Show run details
# RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

Get the details of the HyperDrive runs - what was best?

In [ ]:
import joblib
# Get your best run and save the model from that run.

hd_best_run = hyperdrive_run.get_best_run_by_primary_metric()
hd_best_model = hd_best_run.register_model(model_name="hd_best_model" , model_path="./")

best_run_metrics = hd_best_run.get_metrics()
parameter_values = hd_best_run.get_details()
best_run_filenames = hd_best_run.get_file_names()


print("Best Run Id: ", hd_best_run.id)
print("Best run metrics: ", best_run_metrics)
print("Accuracy: ", best_run_metrics["Accuracy"])
print("Best run details ", parameter_values)
print("Parameters: ", parameter_values["runDefinition"]["arguments"])

Now, we will run the data through AutoML to find out its suggestion on best model.

We begin by setting up the data source - CSV file containing the bank marketing data.

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

csv_url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'    

# Create a TabularDataset using TabularDatasetFactory
ds = TabularDatasetFactory.from_delimited_files(path=csv_url)

The following code registers the dataset in the workspace for easy reuse.

In [ ]:
# Register the dataset in the workspace for reuse
ds= ds.register(workspace=ws,
                name='BankMarketing Dataset',
                description='Bank marketing data from UCI Machine Learning Repository',
                create_new_version=True)

Call the clean_data method from the train.py file to massage the data to make usable.

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

Setup the AutoML job with the configuration parameters.

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target = cpu_cluster,
    experiment_timeout_minutes=15,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    enable_onnx_compatible_models=True,
    n_cross_validations=2)

Run the AutoML job - let us see what it says is the best model.

In [2]:
# Submit your automl run

remote_run = exp.submit(automl_config, show_output=False)
remote_run.wait_for_completion()

Retrieve stats/info from AutoML job on what is found was the best model.

In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = remote_run.get_output()

print(best_run)
print(fitted_model)

# get_metrics()
# Returns the metrics
print("Best run metrics :",best_run.get_metrics())
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())

Cleanup the cluster, delete it to reduce costs.

In [ ]:
# Cleanup environment - delete computer cluster used

cpu_cluster.delete()